In [ ]:
import torch_geometric
from torch_geometric import data
from torch_geometric.utils import from_smiles
import pandas as pd
import torch
import numpy as np


from rdkit import Chem
from rdkit.Chem import AllChem


: 

In [18]:
train_df = pd.read_csv('data/hERg_train.csv')
test_df = pd.read_csv('data/hERg_test.csv')

In [20]:
complex_smiles_train = []
complex_smiles_test = []
train = []
for index, row in train_df.iterrows():
    smiles = row['Smiles']
    data = from_smiles(smiles, with_hydrogen  = False)
    data.y = torch.tensor(row['pIC50']).unsqueeze(-1)

    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)

    status = AllChem.EmbedMolecule(mol)
    if status == -1:
        complex_smiles_train.append(smiles)
        continue
    status = AllChem.UFFOptimizeMolecule(mol)

    mol = Chem.RemoveHs(mol)
    conformer = mol.GetConformer()
    coordinates = conformer.GetPositions()
    coordinates = np.array(coordinates)
    data.pos = torch.tensor(coordinates).float()
    
    train.append(data)

test = []
for index, row in test_df.iterrows():
    smiles = row['Smiles']
    data = from_smiles(smiles, with_hydrogen  = False)
    data.y = torch.tensor(row['pIC50']).unsqueeze(-1)

    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)

    status = AllChem.EmbedMolecule(mol)
    if status == -1:
        complex_smiles_test.append(smiles)
        continue
    status = AllChem.UFFOptimizeMolecule(mol)

    mol = Chem.RemoveHs(mol)
    conformer = mol.GetConformer()
    coordinates = conformer.GetPositions()
    coordinates = np.array(coordinates)
    data.pos = torch.tensor(coordinates).float()
    
    test.append(data)

In [27]:
torch.save(train,'data/train_graph')
torch.save(test,'data/test_graph')

In [ ]:
complex_smiles_train

In [15]:

complex_smiles_test

[]

In [28]:
data

Data(x=[50, 9], edge_index=[2, 108], edge_attr=[108, 3], smiles='COc1ncc(-c2ccc(C(=O)O)cc2C)cc1-c1ccc(C(F)(F)F)cc1CN1C(=O)O[C@H](c2cc(C(F)(F)F)cc(C(F)(F)F)c2)[C@@H]1C', y=[1], pos=[50, 3])